# Notebook for the development of module "weather -> photovoltaic output"
### If you develop some new function useful also in other situations, please put it in a dedicated .py file, then import it here

In [128]:
%pip install openmeteo_requests
%pip install requests-cache retry-requests
%pip install pandas
%pip install pvlib
%pip install pgmpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [129]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

#### Set system parameters here

In [130]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 45.9,
	"longitude": 11.9,
	"start_date": "2024-01-01",
	"end_date": "2024-12-31",
	"hourly": ["temperature_2m", "precipitation", "wind_speed_10m", "snowfall", "rain", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "shortwave_radiation", "direct_radiation", "diffuse_radiation", "direct_normal_irradiance", "global_tilted_irradiance", "terrestrial_radiation"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 45.86994552612305°N 11.96202564239502°E
Elevation 308.0 m asl
Timezone b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0 7200 s


#### Weather request start and end time

In [131]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(6).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(7).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(8).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(9).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(10).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(11).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(12).ValuesAsNumpy()
hourly_global_tilted_irradiance = hourly.Variables(13).ValuesAsNumpy()
hourly_terrestrial_radiation = hourly.Variables(14).ValuesAsNumpy()

#### Call to open-weather

In [132]:
hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["snowfall"] = hourly_snowfall
hourly_data["rain"] = hourly_rain
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                          date  temperature_2m  precipitation  wind_speed_10m  \
0    2023-12-31 22:00:00+00:00          5.6415            1.1        6.287130   
1    2023-12-31 23:00:00+00:00          5.3915            0.2        6.519877   
2    2024-01-01 00:00:00+00:00          5.2415            0.1        4.104631   
3    2024-01-01 01:00:00+00:00          6.0915            0.1        4.198285   
4    2024-01-01 02:00:00+00:00          5.6415            0.1        4.024922   
...                        ...             ...            ...             ...   
8779 2024-12-31 17:00:00+00:00          4.7915            0.0        0.402492   
8780 2024-12-31 18:00:00+00:00          4.1415            0.0        0.402492   
8781 2024-12-31 19:00:00+00:00          3.3915            0.0        0.969330   
8782 2024-12-31 20:00:00+00:00          2.8915            0.0        1.049571   
8783 2024-12-31 21:00:00+00:00          2.4915            0.0        0.900000   

      snowfall  rain  cloud

In [137]:
# --- Your Open-Meteo data retrieval code ---
# ... (previous code to get data into hourly_data dictionary) ...

hourly_dataframe = pd.DataFrame(data = hourly_data)

# --- ENSURE THIS LINE IS PRESENT AND EXECUTED ---
hourly_dataframe = hourly_dataframe.set_index("date")
# ---------------------------------------------

print("Hourly DataFrame head AFTER setting index:")
print(hourly_dataframe.head()) # Add this to verify the index is set
print("Hourly DataFrame index type:", type(hourly_dataframe.index)) # Verify index type

# --- End of your data retrieval code ---

# --- Start PVLib Calculation ---
# ... (the rest of the pvlib code should now work correctly) ...

Hourly DataFrame head AFTER setting index:
                           temperature_2m  precipitation  wind_speed_10m  \
date                                                                       
2023-12-31 22:00:00+00:00          5.6415            1.1        6.287130   
2023-12-31 23:00:00+00:00          5.3915            0.2        6.519877   
2024-01-01 00:00:00+00:00          5.2415            0.1        4.104631   
2024-01-01 01:00:00+00:00          6.0915            0.1        4.198285   
2024-01-01 02:00:00+00:00          5.6415            0.1        4.024922   

                           snowfall  rain  cloud_cover  cloud_cover_low  \
date                                                                      
2023-12-31 22:00:00+00:00       0.0   1.1        100.0             83.0   
2023-12-31 23:00:00+00:00       0.0   0.2        100.0             40.0   
2024-01-01 00:00:00+00:00       0.0   0.1         98.0             92.0   
2024-01-01 01:00:00+00:00       0.0   0.1        

In [136]:
# --- Start PVLib Calculation ---
import pvlib
from pvlib.location import Location
from pvlib.pvsystem import PVSystem, retrieve_sam
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.modelchain import ModelChain
import numpy as np # Add numpy import if not already there

print("\n--- Starting PVLib Calculation ---")

# 1. Prepare Weather Data DataFrame for pvlib
weather_df = hourly_dataframe.copy()
weather_df.rename(columns={
    'shortwave_radiation': 'ghi',         # Global Horizontal Irradiance
    'diffuse_radiation': 'dhi',           # Diffuse Horizontal Irradiance
    'direct_normal_irradiance': 'dni',    # Direct Normal Irradiance
    'temperature_2m': 'temp_air',       # Ambient Air Temperature
    'wind_speed_10m': 'wind_speed',       # Wind Speed
}, inplace=True)

# Ensure required columns exist
required_cols = ['ghi', 'dhi', 'dni', 'temp_air', 'wind_speed']
missing_cols = [col for col in required_cols if col not in weather_df.columns]
if missing_cols:
    raise ValueError(f"Missing required weather columns for PVLib: {missing_cols}")

# Optional: Handle potential NaN values more robustly
if weather_df[required_cols].isnull().values.any():
     print("Warning: NaN values detected in required weather data columns. Imputing with ffill and 0.")
     # Example: Forward fill first, then fill remaining (usually at start) with 0
     weather_df.ffill(inplace=True)
     weather_df.fillna(0, inplace=True)

# --- DEBUG: Inspect weather data just before model run ---
print("\nWeather Data Input to ModelChain (first 5 rows):")
print(weather_df[required_cols].head())
print("\nWeather Data Input Summary:")
print(weather_df[required_cols].describe())
# Check for any negative irradiance values after potential NaN filling
print(f"Any negative GHI/DHI/DNI? GHI: {(weather_df['ghi'] < 0).any()}, DHI: {(weather_df['dhi'] < 0).any()}, DNI: {(weather_df['dni'] < 0).any()}")
# --- END DEBUG ---


# 2. Define Location
latitude = response.Latitude()
longitude = response.Longitude()
altitude = response.Elevation()
tz_bytes = response.Timezone()
tz = tz_bytes.decode('utf-8') if isinstance(tz_bytes, bytes) else tz_bytes

# ... (previous code) ...

location = Location(latitude=latitude, longitude=longitude, altitude=altitude, tz=tz) # Use the decoded string

# --- DEBUG: Check Solar Position for a daytime hour ---
# Pick a timestamp from the middle of a summer day (e.g., July 15th, 1 PM local time)
# Find the corresponding UTC time in your index if needed, or create one
try:
    # Find index for roughly July 15th, 13:00 Europe/Berlin time
    # Berlin is UTC+1/UTC+2 (DST). Let's target 11:00 or 12:00 UTC on July 15th.
    test_time_utc = pd.Timestamp('2024-07-15 12:00:00', tz='UTC')
    if test_time_utc in weather_df.index:
        print(f"\nChecking solar position for: {test_time_utc.tz_convert(tz)}")
        solpos = location.get_solarposition(test_time_utc)
        print(solpos)
        # Also check the irradiance data at this specific time
        print("\nWeather data at this time:")
        print(weather_df.loc[test_time_utc, required_cols])
    else:
        print(f"\nCould not find exact test timestamp {test_time_utc} in index. Index starts: {weather_df.index.min()}, ends: {weather_df.index.max()}")
        # Let's try the middle index value as a proxy
        middle_index_time = weather_df.index[len(weather_df)//2]
        print(f"\nChecking solar position for middle index time: {middle_index_time.tz_convert(tz)}")
        solpos = location.get_solarposition(middle_index_time)
        print(solpos)
        print("\nWeather data at this middle time:")
        print(weather_df.loc[middle_index_time, required_cols])

except Exception as e:
    print(f"\nError during solar position debug check: {e}")
# --- END DEBUG ---


# 3. Define System Parameters (CRITICAL: Choose appropriate components!)
# (Rest of the system definition remains the same as before)
# ... rest of the script ...

# 3. Define System Parameters
sandia_modules = retrieve_sam('SandiaMod')
cec_inverters = retrieve_sam('cecinverter')

module_name = 'Canadian_Solar_Inc__CS6K_275M'
inverter_name = 'SMA_America__SB7000TL_US__240V_'

if module_name not in sandia_modules:
     print(f"Warning: Module '{module_name}' not found in Sandia database. Trying CEC...")
     cec_modules = retrieve_sam('CECMod')
     module_name_cec = 'Canadian_Solar_Inc__CS6X_300M'
     if module_name_cec in cec_modules:
          print(f"Using CEC module: {module_name_cec}")
          module_parameters = cec_modules[module_name_cec]
          module_db = cec_modules
     else:
          raise KeyError(f"Example modules not found in Sandia or CEC databases. Please select an available module.")
else:
     module_parameters = sandia_modules[module_name]
     module_db = sandia_modules

if inverter_name not in cec_inverters:
     print(f"Warning: Inverter '{inverter_name}' not found in CEC database. Trying alternative...")
     inverter_name = 'SMA_America__SB5000TL_US_22__240V_'
     if inverter_name not in cec_inverters:
          raise KeyError(f"Example inverters not found in CEC database. Please select an available inverter.")
     else:
          inverter_parameters = cec_inverters[inverter_name]
else:
     inverter_parameters = cec_inverters[inverter_name]

surface_tilt = 30
surface_azimuth = 180
temp_params = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

modules_per_string = 10
strings_per_inverter = 2

system = PVSystem(surface_tilt=surface_tilt,
                  surface_azimuth=surface_azimuth,
                  module_parameters=module_parameters,
                  inverter_parameters=inverter_parameters,
                  temperature_model_parameters=temp_params,
                  modules_per_string=modules_per_string,
                  strings_per_inverter=strings_per_inverter)

# 5. Create ModelChain object
mc = ModelChain(system, location,
                aoi_model="physical",
                spectral_model="no_loss")

print(f"\nPV System Configuration:")
print(f"- Module: {module_parameters.Name if hasattr(module_parameters, 'Name') else module_name}") # Use .Name attribute
print(f"- Inverter: {inverter_parameters.Name if hasattr(inverter_parameters, 'Name') else inverter_name}")
print(f"- Modules per String: {modules_per_string}")
print(f"- Strings per Inverter: {strings_per_inverter}")
print(f"- Tilt: {surface_tilt} deg, Azimuth: {surface_azimuth} deg")


# 6. Run the Simulation
print("\nRunning model...")
mc.run_model(weather=weather_df)
print("Model run complete.")

# --- DEBUG: Inspect intermediate results ---
# Removed the incorrect mc.results.poa_global line
# print("\nPlane of Array Irradiance (first 5 rows):") # This line was removed
# print(mc.results.poa_global.head())                 # This line was removed
print("\nEffective Irradiance (first 5 rows):")
print(mc.results.effective_irradiance.head())
print("\nCell Temperature (first 5 rows):")
print(mc.results.cell_temperature.head())
print("\nDC Power Output (first 5 rows):")
print(mc.results.dc.head())
# --- END DEBUG ---


# 7. Analyze Results
print("\n--- Simulation Results ---")
print("Calculated AC Power Output (first 5 rows):")
print(mc.results.ac.head())

hourly_dataframe['ac_power_calculated_watts'] = mc.results.ac

# Check for positive AC power before calculating sum
positive_ac = mc.results.ac[mc.results.ac > 0]
if not positive_ac.empty:
    # Calculate interval in hours
    if weather_df.index.freq:
        interval_hours = weather_df.index.freq.total_seconds() / 3600
    else:
        # Estimate interval if freq is not set (less reliable)
        interval_hours = (weather_df.index[1] - weather_df.index[0]).total_seconds() / 3600
        print(f"Warning: Inferring interval as {interval_hours} hours.")

    total_energy_kwh = mc.results.ac.sum() * interval_hours / 1000 # Wh to kWh
    print(f"\nTotal Estimated AC Energy Production for the period: {total_energy_kwh:.2f} kWh")
else:
    # If the sum is negative or zero, it means only losses or no production
    total_energy_kwh = mc.results.ac.sum() * (hourly.Interval() / 3600) / 1000 # Calculate anyway for consistency
    print(f"\nNo significant positive AC power generated. Calculated sum (likely losses): {total_energy_kwh:.2f} kWh")


--- Starting PVLib Calculation ---

Weather Data Input to ModelChain (first 5 rows):
   ghi  dhi  dni  temp_air  wind_speed
0  0.0  0.0  0.0    5.6415    6.287130
1  0.0  0.0  0.0    5.3915    6.519877
2  0.0  0.0  0.0    5.2415    4.104631
3  0.0  0.0  0.0    6.0915    4.198285
4  0.0  0.0  0.0    5.6415    4.024922

Weather Data Input Summary:
               ghi          dhi          dni     temp_air   wind_speed
count  8784.000000  8784.000000  8784.000000  8784.000000  8784.000000
mean    152.170197    57.710041   180.704193    12.886805     4.572430
std     221.849899    78.861473   260.132019     8.149145     2.412876
min       0.000000     0.000000     0.000000    -5.908500     0.000000
25%       0.000000     0.000000     0.000000     6.491500     2.817446
50%       6.000000     5.000000     0.000000    12.841500     4.198285
75%     270.250000   102.000000   357.424873    19.141499     5.904439
max     876.000000   393.000000   885.002197    32.041500    18.584511
Any negative

/home/gian/anaconda3/lib/python3.12/site-packages/pvlib/irradiance.py:848: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  poa_isotropic = np.maximum(dhi * term1 * term2, 0)
/home/gian/anaconda3/lib/python3.12/site-packages/pvlib/irradiance.py:849: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  poa_circumsolar = np.maximum(dhi * (AI * Rb), 0)
/home/gian/anaconda3/lib/python3.12/site-packages/pvlib/irradiance.py:849: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  poa_circumsolar = np.maximum(dhi * (AI * Rb), 0)
/home/gian/anaconda3/lib/python3.12/site-packages/pvlib/irradiance.py:496: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  poa_diffuse = poa_sky_diffuse +